# Quick Conversations With the Companies House API

This notebook describes a basic converational UI for working with the UK Companies House API.

To work with the API, you will need to get an API key: 




In [ ]:
CH_API_TOKEN=''

In [101]:
import urllib2, base64, json
from urllib import urlencode
from time import sleep

def url_nice_req(url,t=300):
    try:
        return urllib2.urlopen(url)
    except urllib2.HTTPError, e:
        if e.code == 429:
            print("Overloaded API, resting for a bit...")
            sleep(t)
            return url_req(url)

In [150]:
#Inspired by http://stackoverflow.com/a/2955687/454773
def ch_request(CH_API_TOKEN,url,args=None):
    if args is not None:
        url='{}?{}'.format(url,urlencode(args))
    request = urllib2.Request(url)
    # You need the replace to handle encodestring adding a trailing newline 
    # (https://docs.python.org/2/library/base64.html#base64.encodestring)
    base64string = base64.encodestring('%s:' % (CH_API_TOKEN)).replace('\n', '')
    request.add_header("Authorization", "Basic %s" % base64string)   
    result = url_nice_req(request)

    #This is too hacky - need to see why it fails if it does
    if result is None:
        print('Oops: {}, {}'.format(url,result))
        return None
        
    j=json.loads(result.read())        
        
    return j

In [13]:
def ch_searchCompanies(q,n=50,start_index='',typ=None):
    #typ: exact, contains, None,
    url= 'https://api.companieshouse.gov.uk/search/companies'
    properties={'q':q,'items_per_page':n,'start_index':''}
    c=ch_request(CH_API_TOKEN,url,properties)
    
    if c is None:
        return c
    
    if typ=='contains':
        c['items']=[i for i in c['items'] if q.lower() in i['title'].lower()]
    elif typ=='exact':
        c['items']=[i for i in c['items'] if q.lower() == i['title'].lower()]
    return c

In [ ]:
ch_searchCompanies('Manchester United',5,typ='contains')

In [16]:
def ch_getCompany(cn):
    url="https://api.companieshouse.gov.uk/company/{}".format(cn)
    return ch_request(CH_API_TOKEN,url)

In [18]:
ch_getCompany('08324083')

{u'accounts': {u'accounting_reference_date': {u'day': u'31', u'month': u'07'},
  u'last_accounts': {u'made_up_to': u'2015-07-31', u'type': u'small'},
  u'next_due': u'2017-04-30',
  u'next_made_up_to': u'2016-07-31',
  u'overdue': False},
 u'annual_return': {u'last_made_up_to': u'2015-12-10',
  u'next_due': u'2017-01-07',
  u'next_made_up_to': u'2016-12-10',
  u'overdue': False},
 u'can_file': True,
 u'company_name': u'FUTURELEARN LIMITED',
 u'company_number': u'08324083',
 u'company_status': u'active',
 u'date_of_creation': u'2012-12-10',
 u'etag': u'4565707d8be4178f121de69373319c1f2787e74d',
 u'has_been_liquidated': False,
 u'has_charges': False,
 u'has_insolvency_history': False,
 u'jurisdiction': u'england-wales',
 u'last_full_members_list_date': u'2015-12-10',
 u'registered_office_address': {u'address_line_1': u'Chambers Building',
  u'address_line_2': u'Walton Hall',
  u'locality': u'Milton Keynes',
  u'po_box': u'77',
  u'postal_code': u'MK7 6BT'},
 u'sic_codes': [u'85590'],
 u'

In [19]:
def ch_getCompanyAddress(cn):
    url="https://api.companieshouse.gov.uk/company/{}/registered-office-address".format(cn)
    return ch_request(CH_API_TOKEN,url)

In [20]:
ch_getCompanyAddress('08324083')

{u'address_line_1': u'Chambers Building',
 u'address_line_2': u'Walton Hall',
 u'locality': u'Milton Keynes',
 u'po_box': u'77',
 u'postal_code': u'MK7 6BT'}

In [21]:
def ch_getCompanyInsolvency(cn):
    url="https://api.companieshouse.gov.uk/company/{}/insolvency".format(cn)
    return ch_request(CH_API_TOKEN,url) #May return None

In [ ]:
ch_getCompanyInsolvency('08324083')

In [22]:
def ch_getCompanyCharges(cn,n=50,start_index=''):
    url="https://api.companieshouse.gov.uk/company/{}/charges".format(cn)
    properties={'items_per_page':n,'start_index':''}
    c=ch_request(CH_API_TOKEN,url,properties)
    return c #May return None

In [ ]:
#ch_getCompanyCharges('08324083')

In [130]:
def ch_getCompanyOfficers(cn,typ='all',role='all'):
    #typ: all, current, previous
    url="https://api.companieshouse.gov.uk/company/{}/officers".format(cn)
    co=ch_request(CH_API_TOKEN,url)
    
    if co is None: return co
    
    if typ=='current':
        co['items']=[i for i in co['items'] if 'resigned_on' not in i]
        #should possibly check here that len(co['items'])==co['active_count'] ?
    elif typ=='previous':
        co['items']=[i for i in co['items'] if 'resigned_on' in i]
    if role!='all':
        co['items']=[i for i in co['items'] if role==i['officer_role']]
    return co

In [ ]:
ch_getCompanyOfficers('08324083',typ='previous',role='director')

In [136]:
def ch_getAppointments(slug,location=None,typ='all',role='all'):
    if len(slug.split('/'))==1:
        slug='/officers/{}/appointments'.format(slug)
    url= 'https://api.companieshouse.gov.uk{}'.format(slug)
    a=ch_request(CH_API_TOKEN,url)

    if a is None: return None
    
    if location is not None:
        a['items']=[i for i in a['items'] if location.lower() in i['address']['locality'].lower()]
    if typ=='current':
        a['items']=[i for i in a['items'] if 'resigned_on' not in i]
        a['items']=[i for i in a['items'] if 'company_status' in i['appointed_to'] and i['appointed_to']['company_status'] == 'active']
        #should possibly check here that len(co['items'])==co['active_count'] ?
    elif typ=='previous':
        a['items']=[i for i in a['items'] if 'resigned_on' in i]
    elif typ=='dissolved':
        a['items']=[i for i in a['items'] if 'company_status' in i['appointed_to'] and i['appointed_to']['company_status'] == 'dissolved']

    if role!='all':
        a['items']=[i for i in a['items'] if role==i['officer_role']]
    return a

In [ ]:
ch_getAppointments('/officers/Tyk3KEbbpwXQ0RJ-h7fRJCf08B4/appointments',typ='dissolved')

In [137]:
def ch_searchOfficers(name,n=50,start_index='',company='',exact=None):
    url= 'https://api.companieshouse.gov.uk/search/officers'
    properties={'q':name,'items_per_page':n,'start_index':start_index} 
    o=ch_request(CH_API_TOKEN,url,properties)
    
    if o is None: return o
    
    if exact=='forename':
        #This isn't right eg double barrelled surnames
        s=name.lower().split(' ')
        o['items'] = [i for i in o['items'] if i['title'].lower().split(' ')[0]==s[0] and i['title'].lower().split(' ')[-1]==s[-1]]
    elif exact=='fullname':
        o['items'] = [i for i in o['items'] if i['title'].lower()==name.lower()]
    if company != '':
        for p in o['items']:
            p['items'] = [i for i in ch_getAppointments(p['links']['self'])['items'] if company.lower() in i['appointed_to']['company_name'].lower()]
        o['items'] = [i for i in o['items'] if len(i['items'])]
    return o

In [ ]:
ch_searchOfficers('Bernard Ecclestone',n=5,company='Formula One')

In [ ]:
ch_searchOfficers('Bernard Ecclestone',n=10,exact='forename')

In [ ]:
ch_searchOfficers('Bernard Ecclestone',n=10)

In [ ]:
#TO DO
# search for companies that a particular person was a director of at a particular time

In [26]:
def ch_disqualifiedOfficer(slug):
    url="https://api.companieshouse.gov.uk{}".format(slug)
    return ch_request(CH_API_TOKEN,url)

def ch_disqualifiedNaturalOfficer(o):
    url="https://api.companieshouse.gov.uk/disqualified-officers/natural/{}".format(o)
    return ch_request(CH_API_TOKEN,url)

def ch_disqualifiedCorporateOfficer(o):
    url='https://api.companieshouse.gov.uk/disqualified-officers/corporate/{}'.format(o)
    return ch_request(CH_API_TOKEN,url)

In [27]:
def ch_searchDisQualifiedOfficers(q,n=5,start_index='',company='',locality=''):
    url= 'https://api.companieshouse.gov.uk/search/disqualified-officers'
    properties={'q':q,'items_per_page':n,'start_index':start_index} 
    o=ch_request(CH_API_TOKEN,url,properties)
    if company != '':
        o['items'] = [j for j in o['items'] for i in ch_disqualifiedOfficer(j['links']['self'])['disqualifications'] for k in i['company_names'] if company.lower() in k.lower()]
    if locality!='':
        o['items'] = [j for j in o['items'] if locality.lower() in j['address']['locality'].lower()]
    return o

In [ ]:
ch_searchDisQualifiedOfficers('Smith',3, locality="Liverpool")

In [ ]:
ch_searchDisQualifiedOfficers('Smith',50,locality='Southampton')

In [ ]:
ch_searchDisQualifiedOfficers('Smith',3,company='land')

In [ ]:
ch_disqualifiedOfficer('/disqualified-officers/natural/Sdiyn47HQMtcH4T1l0MmRlw6kH8')

The Companies House API doesn't provide a very powerful search facility, but we can finesse our own by loading the scattergun results from the Companies House API into an in memory database and then querying again.

In [28]:
def dirCoSeeds(dirseeds,typ='all',role='all'):
    ''' Find companies associated with dirseeds '''
    coseeds=[]
    for d in dirseeds:
        for c in ch_getAppointments(d,typ=typ,role=role)['items']:
            coseeds.append(c['appointed_to']['company_number'])
    return coseeds

In [ ]:
dirseeds=[]
for d in ch_searchOfficers('Anthony Charles Lynton Blair',n=10,exact='forename')['items']:
    dirseeds.append(d['links']['self'])
    
coseeds=dirCoSeeds(dirseeds,typ='current',role='director')
coseeds

In [37]:
!rm example.db

In [46]:
import sqlite3
db=None
memDB=":memory:"
tmpDB='example.db'
if db in locals():
    db.close()

def init_db(tmpDB=':memory:'):
    db = sqlite3.connect(tmpDB)
    c = db.cursor()

    for drop in ['directorslite','companieslite','codirs','coredirs','singlecos']:
        c.execute('''drop table if exists {}'''.format(drop))

    c.execute('''create table directorslite
             (dirnum text primary key,
              dirdob integer,
              dirname text)''')

    c.execute('''create table companieslite
             (conum text primary key,
              costatus text,
              coname text)''')

    c.execute('''create table codirs
             (conum text,
              dirnum text,
              typ text,
              status text)''')

    c.execute('''create table coredirs
             (dirnum text)''')

    c.execute('''create table singlecos
             (conum text,
              coname text)''')
    return c

cosdone=[]
cosparsed=[]
dirsdone=[]
dirsparsed=[]
codirsdone=[]

In [138]:
#coseeds=f1seeds#[]

apicount=600

def apiNice():
    global apicount
    apicount=apicount-1
    if apicount<0:
        print('Giving the api a rest')
        apicount=600
        sleep(250)
        

def updateOnCo(seed,typ='current',role='director'):
    print('harvesting {}'.format(seed))
    
    #apiNice()
    o=ch_getCompanyOfficers(seed,typ=typ,role=role)['items']
    x=[{'dirnum':p['links']['officer']['appointments'].strip('/').split('/')[1],
          'dirdob':p['date_of_birth']['year'] if 'date_of_birth' in p else None,
          'dirname':p['name']} for p in o]
    z=[]
    for y in x:
        if y['dirnum'] not in dirsdone:
            z.append(y)
            dirsdone.append(y['dirnum'])
        if isinstance(z, dict): z=[z]
    print('Adding {} directors'.format(len(z)))
    c.executemany('INSERT INTO directorslite (dirnum, dirdob,dirname)'
                     'VALUES (:dirnum,:dirdob,:dirname)', z)
    for oo in [i for i in o if i['links']['officer']['appointments'].strip('/').split('/')[1] not in dirsparsed]:
        oid=oo['links']['officer']['appointments'].strip('/').split('/')[1]
        print('New director: {}'.format(oid))
        #apiNice()
        ooo=ch_getAppointments(oid,typ=typ,role=role)
        if ooo is None: continue
            
        #apiNice()
        #Play nice with the api
        sleep(0.5)
        #add company details
        x=[{'conum':p['appointed_to']['company_number'],
          'costatus':p['appointed_to']['company_status'] if 'company_status' in p['appointed_to'] else '',
          'coname':p['appointed_to']['company_name'] if 'company_name' in p['appointed_to'] else ''} for p in ooo['items']]
        z=[]
        for y in x:
            if y['conum'] not in cosdone:
                z.append(y)
                cosdone.append(y['conum'])
        if isinstance(z, dict): z=[z]
        print('Adding {} companies'.format(len(z)))
        c.executemany('INSERT INTO companieslite (conum, costatus,coname)'
                     'VALUES (:conum,:costatus,:coname)', z)
        for i in x:cosdone.append(i['conum'])
        #add company director links
        
        dirnum=ooo['links']['self'].strip('/').split('/')[1]
        x=[{'conum':p['appointed_to']['company_number'],'dirnum':dirnum,
            'typ':'current','status':'director'} for p in ooo['items']]
        c.executemany('INSERT INTO codirs (conum, dirnum,typ,status)'
                     'VALUES (:conum,:dirnum,:typ,:status)', x)
        print('Adding {} company-directorships'.format(len(x)))
        dirsparsed.append(oid)
    cosparsed.append(seed)



In [164]:
#Get director details
OUW='01260275'
OUWWL='03221926'
OUSB='03030735'

#Tony Blair
#Firerush Ventures 06901376
#Windrush VEntures 06397276
FIRERUSH='06901376'
WINDRUSH='06397276'

MEEJA='04480425'

GOOGLEUK='03977902'

ARSENAL_FOOTBALL_COMPANY='00109244'

G4SF='09918291'

TESCO='00445790'
TESCOHOLDINGS='00243011'
TAPESILVER='05205362'

ACADEMIES_ENTERPRISE_TRUST='06625091'
HARRIS_FEDERATION='06228587'
UNITED_LEARNING_TRUST='04439859'
REACH2_ACADEMIES_TRUST='08452281'
OASIS_COMMUNITY_LEARNING='05398529'

HALEWOOD_ACADEMY='07909397'

F1='04169896'
F1DIGMEDIA='08915039'

COSWORTH='05177945'
PRODRIVE='01777085'
GINETTA='02744760'
CAPSICUM_CARLIN='07047095'
ARDEN_MOTORSPORT='03146450'

MOTORSPORT_VISION='03446418'
BARC='00516639'
BRDC='01773946'

WILLIAMS_GP_HOLDINGS='07475805'
MCLAREN_GP='04352861'
RED_BULL_RACING='03120645'
MANOR_GP='06661964'
MERCEDES_BENZ_GP='00787446'
FORCE_INDIA_F1='03660294'

IRON_MAIDEN_LLP='OC335408'

RADIOHEAD_TRADEMARK='03408040'

STEREOPHONICS_LTD='03225249'

BLUR_TOURING='OC399569'

THE_WHO_GROUP='00922303'

LIBERTY_HOUSE='04272220'

WORKING_TITLE='05839609'

EMI='00053317'

PURPLE_MANAGEMENT='01066855'
IAN_GILLAN_MUSIC='01098591'

coseeds=[LIBERTY_HOUSE]

In [22]:
coseeds=[co['company_number'] for co in ch_searchCompanies('Manchester United',5,typ='contains')['items']]

In [106]:
ch_searchCompanies('working title',5,typ='contains')

{u'items': [{u'address': {u'address_line_1': u'Blackburn House',
    u'address_line_2': u'Crouch Street',
    u'locality': u'Colchester',
    u'postal_code': u'CO3 3HH',
    u'region': u'Essex'},
   u'company_number': u'05839609',
   u'company_status': u'active',
   u'company_type': u'ltd',
   u'date_of_creation': u'2006-06-07',
   u'description': u'05839609 - Incorporated on  7 June 2006',
   u'description_identifier': [u'incorporated-on'],
   u'kind': u'searchresults#company',
   u'links': {u'self': u'/company/05839609'},
   u'matches': {u'title': [1, 7, 9, 13]},
   u'snippet': u'Blackburn House, Crouch Street, Colchester, Essex, CO3 3HH',
   u'title': u'WORKING TITLE LIMITED'},
  {u'address': {u'address_line_1': u'57-63 North Parade',
    u'locality': u'Bradford',
    u'postal_code': u'BD1 3JH',
    u'region': u'West Yorkshire'},
   u'company_number': u'09274597',
   u'company_status': u'dissolved',
   u'company_type': u'ltd',
   u'date_of_cessation': u'2016-04-05',
   u'date_of_cre

In [171]:
!rm example.db
cosdone=[]
cosparsed=[]
dirsdone=[]
dirsparsed=[]
codirsdone=[]
c=init_db('example.db')

In [172]:
coseeds=['00841344']

In [173]:
typ='current'
#Need to handle director or LLP Designated Member
role='all'
for seed in coseeds:
    updateOnCo(seed,typ=typ,role=role)
c.executemany('INSERT INTO coredirs (dirnum) VALUES (?)', [[d] for d in dirsparsed])

seeder_roles=['Finance Director']
#for dirs in seeded_cos, if dir_role is in seeder_roles then do a second seeding based on their companies
#TO DO
print('done...')
depth=0

harvesting 00841344
Adding 4 directors
New director: w3uRV-ba1aX6PH6wFf8ZRJ79wdc
Adding 4 companies
Adding 4 company-directorships
New director: bcWhTORHkNrxzTUB17JRzeUlTqE
Adding 1 companies
Adding 1 company-directorships
New director: U5zvMYMhGjq7xhgFr0y56A5V95k
Adding 0 companies
Adding 1 company-directorships
New director: b-CCU8mD1pO6F6vB9f-5hp-r2-E
Adding 0 companies
Adding 1 company-directorships
done...


In [174]:
seeder=True
oneDirSeed=True
#typ='current'
#role='director'
maxdepth=3
#relaxed=0
while depth<maxdepth:
    print('---------------\nFilling out level - {}...'.format(depth))
    if seeder and depth==0:
        #Another policy would be dive on all companies associated w/ dirs of seed
        #In which case set the above test to depth==0
        tofetch=[u[0] for u in c.execute(''' SELECT DISTINCT conum from codirs''')]
    else:
        duals=c.execute('''SELECT cd1.conum as c1,cd2.conum as c2, count(*) FROM codirs AS cd1
                        LEFT JOIN codirs AS cd2 
                        ON cd1.dirnum = cd2.dirnum AND cd1.dirnum
                        WHERE cd1.conum < cd2.conum GROUP BY c1,c2 HAVING COUNT(*)>1
                        ''')
        tofetch=[x for t in duals for x in t[:2]]
        #The above has some issues. eg only 1 director is required, and secretary IDs are unique to company
        #Maybe need to change logic so if two directors OR company just has one director?
        #if relaxed>0:
        #    print('Being relaxed {} at depth {}...'.format(relaxed,depth))
        #    duals=c.execute('''SELECT cd.conum as c1,cl.coname as cn, count(*) FROM codirs as cd JOIN companieslite as cl 
        #                 WHERE cd.conum= cl.conum GROUP BY c1,cn HAVING COUNT(*)=1
        #                ''')
        #    tofetch=tofetch+[x[0] for x in duals]
        #    relaxed=relaxed-1
    if depth==0 and oneDirSeed:
        #add in companies with a single director first time round
        sco=[]
        for u in c.execute('''SELECT DISTINCT cd.conum, cl.coname FROM codirs cd  JOIN companieslite cl ON
                                cd.conum=cl.conum'''):
            #apiNice()
            o=ch_getCompanyOfficers(u[0],typ=typ,role=role)
            if len(o['items'])==1 or u[0]in coseeds:
                sco.append({'conum':u[0],'coname':u[1]})
                tofetch.append(u[0])
        c.executemany('INSERT INTO singlecos (conum,coname) VALUES (:conum,:coname)', sco)
    #TO DO: Another stategy might to to try to find the Finance Director or other names role and seed from them?
    
    #Get undone companies
    print('To fetch: ',[u for u in tofetch if u not in cosparsed])
    for u in [x for x in tofetch if x not in cosparsed]:
            updateOnCo(u,typ=typ,role=role)
            cosparsed.append(u)
            #play nice
            #apiNice()
    depth=depth+1
    #Parse companies

---------------
Filling out level - 0...
('To fetch: ', [u'03815912', u'03459209', u'00989959', u'SC014586'])
harvesting 03815912
Adding 1 directors
New director: Pjx7-ZTd2uR1mqlJ40iNIINKTQA
Adding 0 companies
Adding 1 company-directorships
harvesting 03459209
Adding 3 directors
New director: Ez92urM0DBXEXRMYl_MXrSduVHc
Adding 34 companies
Adding 35 company-directorships
New director: WuFnOHXbXRVLGc2bQGkcrgpiWS4
Adding 18 companies
Adding 31 company-directorships
New director: 6sbSgRBN7vbw1fQrfGXJqGK-fe4
Adding 0 companies
Adding 1 company-directorships
harvesting 00989959
Adding 1 directors
New director: GA8GhuyYjJr3nxNniKeglujJZBg
Adding 12 companies
Adding 30 company-directorships
harvesting SC014586
Adding 1 directors
New director: 1_poRFnoQHfmjeP3K1iEJiNd18o
Adding 1 companies
Adding 2 company-directorships
---------------
Filling out level - 1...
('To fetch: ', [])
---------------
Filling out level - 2...
('To fetch: ', [])


In [175]:
for u in c.execute('''SELECT cl1.coname as c1,cl2.coname as c2, count(*) FROM codirs AS cd1
                        LEFT JOIN codirs AS cd2 JOIN companieslite as cl1 JOIN companieslite as cl2
                        ON cd1.dirnum = cd2.dirnum and cd1.conum=cl1.conum and cd2.conum=cl2.conum
                        WHERE cd1.conum < cd2.conum GROUP BY c1,c2 HAVING COUNT(*)>1'''): print(u)
print('-------')    
#if a co has a singleton dir and that dir is in coredirs, then join cos for that coredir
for u in c.execute('''SELECT cl1.coname as c1,cl2.coname as c2 FROM codirs AS cd1
                        LEFT JOIN codirs AS cd2 JOIN singlecos as cl1 JOIN singlecos as cl2
                        ON cd1.dirnum = cd2.dirnum and cd1.conum=cl1.conum and cd2.conum=cl2.conum
                        WHERE cd1.conum < cd2.conum'''):
    print(u)
    #for x in c.execute('''SELECT dirnum '''.format()):
    #    if  print(u)
#for u in c.execute('''SELECT cd.conum as c1,cl.coname as cn, count(*) FROM codirs as cd JOIN companieslite as cl 
#                         WHERE cd.conum= cl.conum GROUP BY c1,cn HAVING COUNT(*)=1
#                        '''): print(u)

#TO DO
#for gephi, output with Source, Target and perhaps Weight (no. of co-dirs)

(u'AVDEL HOLDING LIMITED', u'AVDEL UK LIMITED', 2)
(u'AVDEL HOLDING LIMITED', u'BLACK & DECKER EUROPE', 2)
(u'AVDEL HOLDING LIMITED', u'BLACK & DECKER FINANCE', 2)
(u'AVDEL HOLDING LIMITED', u'MASTERFIX PRODUCTS U.K. LTD', 2)
(u'AVDEL HOLDING LIMITED', u'PIH SERVICES LIMITED', 2)
(u'AVDEL HOLDING LIMITED', u'PIPELINE INDUCTION HEAT LIMITED', 2)
(u'AVDEL HOLDING LIMITED', u'SPIRALOCK OF EUROPE LTD.', 2)
(u'AVDEL HOLDING LIMITED', u'STANLEY BLACK & DECKER INNOVATIONS LIMITED', 2)
(u'AVDEL HOLDING LIMITED', u'STANLEY SECURITY SOLUTIONS (NI) LIMITED', 2)
(u'AVDEL UK LIMITED', u'BLACK & DECKER EUROPE', 2)
(u'AVDEL UK LIMITED', u'BLACK & DECKER FINANCE', 2)
(u'AVDEL UK LIMITED', u'MASTERFIX PRODUCTS U.K. LTD', 2)
(u'AVDEL UK LIMITED', u'PIH SERVICES LIMITED', 2)
(u'AVDEL UK LIMITED', u'PIPELINE INDUCTION HEAT LIMITED', 2)
(u'AVDEL UK LIMITED', u'SPIRALOCK OF EUROPE LTD.', 2)
(u'AVDEL UK LIMITED', u'STANLEY BLACK & DECKER INNOVATIONS LIMITED', 2)
(u'AVDEL UK LIMITED', u'STANLEY SECURITY SOLUT

In [63]:
data=c.execute('''SELECT cl1.coname as Source,cl2.coname as Target, count(*) FROM codirs AS cd1
                        LEFT JOIN codirs AS cd2 JOIN companieslite as cl1 JOIN companieslite as cl2
                        ON cd1.dirnum = cd2.dirnum and cd1.conum=cl1.conum and cd2.conum=cl2.conum
                        WHERE cd1.conum < cd2.conum GROUP BY Source,Target HAVING COUNT(*)>1''')
import csv
with open('output1.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerow(['Source', 'Target'])
    writer.writerows(data)
    
data= c.execute('''SELECT cl1.coname as c1,cl2.coname as c2 FROM codirs AS cd1
                        LEFT JOIN codirs AS cd2 JOIN singlecos as cl1 JOIN singlecos as cl2
                        ON cd1.dirnum = cd2.dirnum and cd1.conum=cl1.conum and cd2.conum=cl2.conum
                        WHERE cd1.conum < cd2.conum''')
with open('output1.csv', 'ab') as f:
    writer = csv.writer(f)
    writer.writerows(data)

In [119]:
data=c.execute('''SELECT cl.coname as Source,dl.dirname as Target, count(*) FROM codirs AS cd
                JOIN companieslite as cl
                JOIN directorslite as dl ON cd.dirnum = dl.dirnum and cd.conum=cl.conum''')
import csv
with open('output2.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerow(['Source', 'Target'])
    writer.writerows(data)


In [ ]:
for u in c.execute(''' select cd.dirname as cl.coname join directorslite as d join companieslite as cl on 
        cd.dirnum=d.dirum and cd.conum=cl.conum where  '''):print(u)

In [170]:
import networkx as nx

G=nx.Graph()

data=c.execute('''SELECT cl.conum as cid, cl.coname as cn, dl.dirnum as did, dl.dirname as dn
FROM codirs AS cd JOIN companieslite as cl JOIN directorslite as dl ON cd.dirnum = dl.dirnum and cd.conum=cl.conum ''')
for d in data:
    G.add_node(d[0], Label=d[1])
    G.add_node(d[2], Label=d[3])
    G.add_edge(d[0],d[2])
nx.write_gexf(G, "deepPurple.gexf")

In [47]:
G.nodes()

[u'04005803',
 u'08809035',
 u'SC001731',
 u'w7rPZmLX1Kq10noDl-0nY7Xwctg',
 u'ZC000201',
 u'oDC8fOOaXjgQ5cGdnQrjJvnOfgw',
 u'02071638',
 u'08124892',
 u'5gDEeqcRYtXLlu4OExETJCmiSYA',
 u'06930140',
 u'Kw6ZC_Fk6J4kr3Ws7qkU3cOQR-s',
 u'07898613',
 u'06675840',
 u'04299687',
 u'07899149',
 u'03868390',
 u'co2na_Q0pmr0QeLgO6pQrekbWdM',
 u'04049010',
 u'04326560',
 u'06955383',
 u'r3SnJW5li6I829bctvw-er4O0v8',
 u'09150715',
 u'03279074',
 u'02778481',
 u'w1qOvSziLFFQyKiSimy0GRHqX5g',
 u'5FWxIlOPARIx-1fkWUR21wnuNPQ',
 u'07541787',
 u'05114014',
 u'-V6DT09dCnU-oV-cF2b5Q6AyCTA',
 u'07879081',
 u'H7vizJ3kXpJRRWnDkVels5Oo78M',
 u'SC418144',
 u's0UVAKHvRcga63Vm6BAYrCb-xcM',
 u'07254495',
 u'iKj9-xgWvlLwZT8FV2RyPbmC6PM',
 u'DvLVvNIKHQWUG_fdnYtvb06dPjc',
 u'cHTjveo0S8VcXeNdKPETVpsuopA',
 u'02820162',
 u'06640309',
 u'cjWQMbx_qOQ3zZD9FAdI-a_NGBc',
 u'IJzxdKXbQ4Q5kqLf8gjbDbC7ZDU',
 u'08059425',
 u'01702397',
 u'03998722',
 u'06239871',
 u'07435679',
 u'04185635',
 u'02798901',
 u'02724038',
 u'0129356

In [54]:
def gPath(c,G,x,y):
    c.execute("SELECT c.conum FROM companieslite c WHERE c.coname=?",[x])
    xn=c.fetchone()
    c.execute("SELECT c.conum FROM companieslite c WHERE c.coname=?",[y])
    yn=c.fetchone()
    for p in nx.shortest_path(G, xn[0],yn[0]):
        print(G.node[p]['Label'])
    return nx.shortest_path(G, xn[0],yn[0])
    
path=gPath(c,G,'THE EMMANUEL SCHOOLS FOUNDATION','UNITED LEARNING TRUST')

THE EMMANUEL SCHOOLS FOUNDATION
ROBSON, Nigel Rhyl
UNITED LEARNING TRUST


In [34]:
!head test.gexf

<gexf version="1.1" xmlns="http://www.gexf.net/1.1draft" xmlns:viz="http://www.gexf.net/1.1draft/viz" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.w3.org/2001/XMLSchema-instance">
  <graph defaultedgetype="undirected" mode="static">
    <attributes class="node" mode="static">
      <attribute id="0" title="name" type="string" />
    </attributes>
    <nodes>
      <node id="04005803" label="04005803">
        <attvalues>
          <attvalue for="0" value="TAUNTON SCHOOL EDUCATIONAL CHARITY" />
        </attvalues>


In [688]:
for u in c.execute('''SELECT * from coredirs'''): print(u)

(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s',)
(u'pUbh50NYJEANkTKi9ADvWxqs22w',)
(u'M4tn3QS93s_crb8c9UAaRr8SnA4',)
(u'MeeK3ZOQPi2M8QSJ53tVEulX_aQ',)
(u'7x5ydDSuaMx71HB8usAouJQpphQ',)
(u'Bxbf_Fv1GuDBBTHbqNZRVcmKSfA',)


In [545]:
for u in c.execute('''SELECT * from directorslite'''): print(u)

(u'szfnholtU7mzYfbeFw6sqP_mMvo', None, u'MOORE, Paul Anthony')
(u'f15C_rkyG6KnW75E6ZUle1Ah1os', 1948, u'ALLAN, John Murray')
(u'6LkwdoJ_Rn10_1Fqf34JHbbGI1c', 1954, u'ARMOUR, Mark Henry')
(u'ckp4z5BSV6woEsgZDTwbwjtln4U', 1959, u'COUSINS, Richard John')
(u'99Q3irUhqx-AIjRaITsuH0dK9xw', 1948, u'GROTE, Byron Elmer')
(u'lRecVAOeUI710jXPzNBvhVQk8fE', 1965, u'LEWIS, David John')
(u't77338rRl1jkPgPtSsqMekiZNJg', 1957, u'OLSSON, Anders Bertil Mikael')
(u'KQ_-Dhro_IPanvxuKtN0qTwOtVA', 1958, u'OPPENHEIMER, Deanna Watson')
(u'5o1GJ8yqpl1blGFV9SRohZVIIPg', 1960, u'STEWART, Alan James Harris')


In [155]:
for u in c.execute('''SELECT * from companieslite'''): print(u)

(u'05202976', u'active', u'RED BULL TECHNOLOGY LIMITED')
(u'03120645', u'active', u'RED BULL RACING LIMITED')


In [88]:
for u in c.execute('''SELECT * from codirs'''): print(u)

(u'06351418', u'FWFjfth2pDDz9yCzltK3QgylN00', u'current', u'director')
(u'03070534', u'FWFjfth2pDDz9yCzltK3QgylN00', u'current', u'director')
(u'05133392', u'FWFjfth2pDDz9yCzltK3QgylN00', u'current', u'director')
(u'05396112', u'FWFjfth2pDDz9yCzltK3QgylN00', u'current', u'director')
(u'07068440', u'FWFjfth2pDDz9yCzltK3QgylN00', u'current', u'director')
(u'07034809', u'FWFjfth2pDDz9yCzltK3QgylN00', u'current', u'director')
(u'07034840', u'FWFjfth2pDDz9yCzltK3QgylN00', u'current', u'director')
(u'06869216', u'FWFjfth2pDDz9yCzltK3QgylN00', u'current', u'director')
(u'06715745', u'FWFjfth2pDDz9yCzltK3QgylN00', u'current', u'director')
(u'06574249', u'FWFjfth2pDDz9yCzltK3QgylN00', u'current', u'director')
(u'05744532', u'FWFjfth2pDDz9yCzltK3QgylN00', u'current', u'director')
(u'03062301', u'FWFjfth2pDDz9yCzltK3QgylN00', u'current', u'director')
(u'04992338', u'FWFjfth2pDDz9yCzltK3QgylN00', u'current', u'director')
(u'04992360', u'FWFjfth2pDDz9yCzltK3QgylN00', u'current', u'director')
(u'049

In [574]:
duels=c.execute('''SELECT cd.conum as c1,cl.coname as cn, count(*) FROM codirs as cd JOIN companieslite as cl 
                         WHERE cd.conum= cl.conum GROUP BY c1,cn HAVING COUNT(*)=1
                        ''')
[x for t in duels for x in t[:1]]

[u'02298419',
 u'02324094',
 u'02834529',
 u'03471126',
 u'03664571',
 u'04007730',
 u'06617598',
 u'07073195',
 u'09248309',
 u'NF004128',
 u'SC501980']

In [779]:
for u in c.execute('''select dirnum from coredirs'''):print(u)

(u'1Ogl5BMpeIh3mMr0L9SAPmVXGL0',)
(u'R5DDeApFVPK6yCeA6K0tUlvRn_Y',)
(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s',)
(u'Bp0ZKnlgPMkXgTaGH-64iDbsiwc',)
(u'P-6bXgs3tIwG125MlTxwPXq-Dr8',)
(u'pUbh50NYJEANkTKi9ADvWxqs22w',)
(u'M4tn3QS93s_crb8c9UAaRr8SnA4',)
(u'MeeK3ZOQPi2M8QSJ53tVEulX_aQ',)
(u'7x5ydDSuaMx71HB8usAouJQpphQ',)
(u'Bxbf_Fv1GuDBBTHbqNZRVcmKSfA',)
(u'1Ogl5BMpeIh3mMr0L9SAPmVXGL0',)
(u'R5DDeApFVPK6yCeA6K0tUlvRn_Y',)
(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s',)
(u'Bp0ZKnlgPMkXgTaGH-64iDbsiwc',)
(u'P-6bXgs3tIwG125MlTxwPXq-Dr8',)
(u'pUbh50NYJEANkTKi9ADvWxqs22w',)
(u'M4tn3QS93s_crb8c9UAaRr8SnA4',)
(u'MeeK3ZOQPi2M8QSJ53tVEulX_aQ',)
(u'7x5ydDSuaMx71HB8usAouJQpphQ',)
(u'Bxbf_Fv1GuDBBTHbqNZRVcmKSfA',)
(u'A9j3-RX5cCIL17ThB6qFoUF3o9U',)
(u'HjbmjWXbG2NcwvpiX4NkXDwqajU',)
(u'K5y4fJVG_tr5ZZrTwTBX0L30CBg',)
(u'qyI4p8BQhdG2cYJg0PWgdUFCjnQ',)
(u'spI8-SMbOF96rmr2RR5v_J3cJdQ',)
(u'FUtcrbinZ6I0ZRdvk0O1A9LUp64',)
(u'7SaFM9AuCxTCK0IBPxjm-0Axv2c',)
(u'LY09KNpILZcq5X1xxm3ih_qsh2Q',)
(u'2C81RT3ixhyXCwOnWFZWPu0Wcao',)
(u'Uq0fKk-3ayH

In [691]:
for u in c.execute('''select dirnum,conum from codirs'''):print(u)

(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'03221606')
(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'03221926')
(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'03221613')
(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'03221609')
(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'08877980')
(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'08324083')
(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'04461492')
(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'04087160')
(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'04461493')
(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'03779976')
(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'04594014')
(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'03825045')
(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'01260275')
(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'02995193')
(u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'03030735')
(u'pUbh50NYJEANkTKi9ADvWxqs22w', u'01260275')
(u'M4tn3QS93s_crb8c9UAaRr8SnA4', u'08324083')
(u'M4tn3QS93s_crb8c9UAaRr8SnA4', u'01260275')
(u'MeeK3ZOQPi2M8QSJ53tVEulX_aQ', u'01260275')
(u'MeeK3ZOQPi2M8QSJ53tVEulX_aQ', u'08324083')
(u'7x5ydDSuaMx71HB8usAouJQpphQ', u'01260275')
(u'Bxbf_Fv1GuDBBTHbqNZRVcmKSfA', u

In [90]:
for u in c.execute('''SELECT * FROM singlecos'''):print(u)

In [694]:
for u in c.execute('''SELECT c.conum as cn, c.coname as cna FROM
codirs as cd JOIN companieslite as c WHERE c.conum=cd.conum and cd.dirnum in (select dirnum from coredirs) GROUP BY cn,cna HAVING COUNT(*)=1 '''):print(u)

(u'02995193', u'OPEN UNIVERSITY BUSINESS SCHOOL LIMITED')
(u'03030735', u'OPEN UNIVERSITY STUDENT BUDGET ACCOUNTS LIMITED')
(u'03221606', u'WORLDWIDE OPEN UNIVERSITY LTD')
(u'03221609', u'OPEN UNIVERSITY EDUCATIONAL ENTERPRISES LIMITED')
(u'03221613', u'OPEN UNIVERSITY INTERNATIONAL LTD')
(u'03221926', u'OPEN UNIVERSITY WORLDWIDE LEARNING LTD')
(u'03779976', u'BLOXHAM SCHOOL LIMITED')
(u'03825045', u'BOOKHIRE LIMITED')
(u'04087160', u'OU PROPERTIES (BRISTOL) LIMITED')
(u'04461492', u'OPEN UNIVERSITY ENTERPRISES LIMITED')
(u'04461493', u'OU PROPERTIES (WALTON DRIVE) LIMITED')
(u'04594014', u'OUEB LIMITED')
(u'08877980', u'GREEN & CHAMBERS LEGAL LIMITED')


In [486]:
duals=c.execute('''SELECT * FROM codirs''')
#duals=c.execute('''select dirnum from coredirs''')
for u in duals: print(u)

(u'03221606', u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'current', u'director')
(u'03221926', u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'current', u'director')
(u'03221613', u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'current', u'director')
(u'03221609', u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'current', u'director')
(u'08877980', u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'current', u'director')
(u'08324083', u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'current', u'director')
(u'04461492', u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'current', u'director')
(u'04087160', u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'current', u'director')
(u'04461493', u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'current', u'director')
(u'03779976', u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'current', u'director')
(u'04594014', u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'current', u'director')
(u'03825045', u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'current', u'director')
(u'01260275', u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'current', u'director')
(u'02995193', u'eJZIFS2nbSHZkvCgtaufbJWaJ_s', u'current', u'director')
(u'030

In [489]:
duals=c.execute('''SELECT * FROM directorslite where dirnum="9Lc-6eIIC21nNFTERPV3aWGaKSQ"''')
#duals=c.execute('''select dirnum from coredirs''')
for u in duals: print(u)

(u'9Lc-6eIIC21nNFTERPV3aWGaKSQ', 1969, u'NELSON, Simon Jonathan')


In [488]:
duals=c.execute('''SELECT * FROM directorslite where dirname="CHEYNE, Brian"''')
#duals=c.execute('''select dirnum from coredirs''')
for u in duals: print(u)

In [417]:
duals=c.execute('''SELECT * FROM directorslite d JOIN companieslite c JOIN codirs as cd where d.dirnum=cd.dirnum and c.conum=cd.conum ''')
for u in duals: print(u)

(u'f15C_rkyG6KnW75E6ZUle1Ah1os', 1948, u'ALLAN, John Murray', u'08762327', u'active', u'WORLDPAY GROUP PLC', u'08762327', u'f15C_rkyG6KnW75E6ZUle1Ah1os', u'current', u'director')
(u'f15C_rkyG6KnW75E6ZUle1Ah1os', 1948, u'ALLAN, John Murray', u'00445790', u'active', u'TESCO PLC', u'00445790', u'f15C_rkyG6KnW75E6ZUle1Ah1os', u'current', u'director')
(u'f15C_rkyG6KnW75E6ZUle1Ah1os', 1948, u'ALLAN, John Murray', u'02756521', u'active', u'LONDON FIRST', u'02756521', u'f15C_rkyG6KnW75E6ZUle1Ah1os', u'current', u'director')
(u'f15C_rkyG6KnW75E6ZUle1Ah1os', 1948, u'ALLAN, John Murray', u'00604574', u'active', u'BARRATT DEVELOPMENTS P L C', u'00604574', u'f15C_rkyG6KnW75E6ZUle1Ah1os', u'current', u'director')
(u'f15C_rkyG6KnW75E6ZUle1Ah1os', 1948, u'ALLAN, John Murray', u'07055485', u'liquidation', u'NATAKATE LTD', u'07055485', u'f15C_rkyG6KnW75E6ZUle1Ah1os', u'current', u'director')
(u'6LkwdoJ_Rn10_1Fqf34JHbbGI1c', 1954, u'ARMOUR, Mark Henry', u'00445790', u'active', u'TESCO PLC', u'00445790', 

In [394]:
duals=c.execute('''SELECT conum FROM codirs
                    ''')
for u in duals: print(u)

(u'08762327',)
(u'00445790',)
(u'02756521',)
(u'00604574',)
(u'07055485',)
(u'00445790',)
(u'08645471',)
(u'02486368',)
(u'01550778',)
(u'07697612',)
(u'03528416',)
(u'06246609',)
(u'00445790',)
(u'00445790',)
(u'00243011',)
(u'00519500',)
(u'00445790',)
(u'00445790',)
(u'SC173198',)
(u'SC173199',)
(u'00445790',)
(u'00243011',)
(u'00519500',)
(u'08629715',)
(u'00445790',)


In [60]:
c.executemany('INSERT INTO directors (id, dob,addr,title,cnt,descr) '
                 'VALUES (:id,:dob,:addr,:title,:cnt,:descr)', x)

In [ ]:
y=[{'conum':q['appointed_to']['company_number'],
          'costatus':q['appointed_to']['company_status'],
          'coname':q['appointed_to']['company_name']} for q in e['items'] ]

In [ ]:
z=[{'conum':,'dirnum':, 'typ':, 'status':}]

In [61]:
for r in c.execute('select * from directors where title="Peter Horrocks" COLLATE NOCASE order by cnt desc'):
    print(r)

(u'/officers/fx_cgCEl1nfMR8pXlcqlemKowxs/appointments', 1960, u'26 Stretton Road, Greenmount, Bury, Lancashire, BL8 4DF', u'Peter HORROCKS', 4, u'Total number of appointments 4 - Born April 1960')
(u'/officers/pHhQOg0gUt8V-8CWgKMx8P3tEdQ/appointments', 1953, u'Orchard Lea, Old Bury Road, Alpheton, Sudbury, Suffolk, United Kingdom, CO10 9BT', u'Peter HORROCKS', 3, u'Total number of appointments 3 - Born October 1953')
(u'/officers/LnKbanK0iJ7Ovo2SOdySBgL8PVM/appointments', 1942, u'Mon Reve, Manchester Road, Blackrod, Bolton, Lancashire, BL6 5LS', u'Peter HORROCKS', 2, u'Total number of appointments 2 - Born January 1942')
(u'/officers/MeeK3ZOQPi2M8QSJ53tVEulX_aQ/appointments', 1959, u'Walton Hall, Milton Keynes, Buckinghamshire, MK7 6AA', u'Peter HORROCKS', 2, u'Total number of appointments 2 - Born October 1959')
(u'/officers/gCQeLQmnBol7dzvRUwSRzcQESvk/appointments', 1945, u'Redmayne Hall, Little Urswick, Ulverston, Cumbria, LA12 0PL', u'Peter HORROCKS', 2, u'Total number of appointme

In [55]:
for r in c.execute("select * from directors where addr LIKE '%Manchester%' order by cnt desc"):
    print(r)

(u'/officers/LnKbanK0iJ7Ovo2SOdySBgL8PVM/appointments', 1942, u'Mon Reve, Manchester Road, Blackrod, Bolton, Lancashire, BL6 5LS', u'Peter HORROCKS', 2, u'Total number of appointments 2 - Born January 1942')


In [26]:
from datetime import date

def dirCompanies(a):
    if not len(a['items']): return
    if 'name' in a: name=a['name']
    else:name=a['title']
    print('{} [{}], {}, has the following appointments:'.format(name,a['links']['self'].split('/')[2],
                                                       date.today().year-a['date_of_birth']['year'] -(date.today().month <a['date_of_birth']['month'] )))
    
    for aa in a['items']:
        print('\t- {}, {} ({}): {} ({})'.format(aa['appointed_to']['company_name'],
                                        aa['address']['locality'] if 'locality' in aa['address'] else '-',
                                    aa['appointed_to']['company_number'],
                                    aa['officer_role'],
                                    aa['appointed_on'] if 'appointed_on' in aa else aa['appointed_before']))


In [81]:
import re

regexp=re.compile(r'.*companies ?(?:is|are) (.*) a director of?')

m=re.match(regexp,'What companies is Peter Horrocks a director of?')
for c in ch_searchOfficers(m.group(1),n=5)['items']:
    dirCompanies(ch_getAppointments(c['links']['self']))

Andrew Peter HORROCKS [YxnxEXxiCUWChbBlnNm5wo-DYqM], 45, has the following appointments:
	- ANDREW HORROCKS LIMITED, Preston, (04841743): director (2003-07-22)
Peter HORROCKS [LnKbanK0iJ7Ovo2SOdySBgL8PVM], 74, has the following appointments:
	- C.HORROCKS & SONS LIMITED, Bolton (00783568): director (1991-06-20)
	- C.HORROCKS & SONS LIMITED, Lancs (00783568): secretary (1991-06-20)
Peter HORROCKS [MeeK3ZOQPi2M8QSJ53tVEulX_aQ], 56, has the following appointments:
	- OPEN UNIVERSITY WORLDWIDE LIMITED, Buckinghamshire (01260275): director (2015-06-11)
	- FUTURELEARN LIMITED, Milton Keynes (08324083): director (2015-06-11)
Peter HORROCKS [gCQeLQmnBol7dzvRUwSRzcQESvk], 70, has the following appointments:
	- HORROCKS AND CROSS LIMITED, Ulverston (02094977): director (1991-12-31)
	- HORROCKS AND CROSS LIMITED, Ulverston (02094977): secretary (1991-12-31)
Peter HORROCKS [pHhQOg0gUt8V-8CWgKMx8P3tEdQ], 62, has the following appointments:
	- P H SOCIAL WORK LIMITED, Sudbury (07477208): director (2

In [90]:
import re
regexp=re.compile(r'Which (.*) is a director of (.*?)\??$')

m=re.match(regexp,'Which michael jagger is a director of Rolling Stones?')
for c in ch_searchOfficers(m.group(1),n=100,company=m.group(2))['items']:
    dirCompanies(c)

In [30]:
import re
regexp=re.compile(r'.*companies in (.*?) (?:is|are) (.*) a director of\?')

m=re.match(regexp,'What companies in Wight is Simon Perry a director of?')

for c in ch_searchOfficers(m.group(2),n=5)['items']:
    dirCompanies(ch_getAppointments(c['links']['self'],location=m.group(1)))

In [28]:
import re
regexp=re.compile(r'Who were the previous directors of (.*?)[\?]?$')

m=re.match(regexp,'Who were the previous directors of Futurelearn?')
for c in ch_searchCompanies(m.group(1),5,typ='contains')['items']:
    print(c['title'])
    for o in ch_getCompanyOfficers(c['company_number'],typ='previous')['items']:
        print('{} (resigned on {})'.format(o['name'],o['resigned_on']))
    print('----')

FUTURELEARN LIMITED
BEAN, Martin George (resigned on 2016-03-04)
CRAMER, Stephen John (resigned on 2013-01-21)
WOODBURN, Alexander Fraser (resigned on 2015-12-18)
----


In [111]:
xx=ch_searchOfficers('Michael Jagger',n=50,company='Formula One')['items'][0]['items']
f1seeds=[i['appointed_to']['company_number'] for i in xx]

[u'04169896',
 u'08915039',
 u'08445563',
 u'01545332',
 u'05731193',
 u'04174493',
 u'07714752',
 u'01867673',
 u'00968180',
 u'08445563',
 u'07714752',
 u'05731193',
 u'01545332',
 u'04174493',
 u'04169896',
 u'03737094']

In [ ]:
def exact_match(items, txt):
    ''' Prioritise the list so that exact matches are first... '''
    pass